Olá, nesse notebook a idéia é, utilizando o PyTorch, resolver um problema de classificação de imagens de tomates e maçãs. 
O dataset utilizado é o Apples or Tomatoes obtido no Kaggle.

Para começar, vamos importar as bibliotecas necessárias e carregar o dataset.
Certifique-se de já ter rodado o pip install requirements.txt do dir root desse projeto.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np

No PyTorch se utiliza **Transforms** para pré-processar imagens. 

O transforms no PyTorch faz parte do *torchvision.transforms*, que é um módulo responsável por pré-processar imagens antes de passá-las para um modelo de deep learning. Ele é essencial para normalização, redimensionamento e aumento de dados (data augmentation).

**Os modelos de deep learning convergem mais rápido se os valores dos pixels forem normalizados.**

In [4]:
# Definição das transformações para normalizar e aumentar os dados
transform = transforms.Compose([
    transforms.Resize((150, 150)),  # Redimensionar para tamanho fixo
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

Agora vamos definir o caminho de diretório das imagens, e em seguida, vincular eles com o transform.

Feita a vinculação vamos verificar as **Classes** que foram criadas, ele separa as classes a partir do nome do diretório, então é importante que as imagens estejam organizadas em pastas com o nome da classe de cada objeto correspondente.

Para verificar, vamos dar um *print* no *train_dataset* e no *test_dataset*, a saída deve ser:

Classes de Treino:  {'apples': 0, 'tomatoes': 1}\
Classes de Teste:  {'apples': 0, 'tomatoes': 1}




In [5]:
# Diretórios das imagens - Caminho Absoluto
train_dir = "C:\\Users\\Matheus\\Documents\\Github Projects\\research_MLs\\projects\\LMs\\dataset\\tomatoes_vs_apples\\train"
test_dir = "C:\\Users\\Matheus\\Documents\\Github Projects\\research_MLs\\projects\\LMs\\dataset\\tomatoes_vs_apples\\test"

# Carregamento dos datasets
train_dataset = ImageFolder(root=train_dir, transform=transform)
test_dataset = ImageFolder(root=test_dir, transform=transform)

# Verificando classes
print("Classes de Treino: ",train_dataset.class_to_idx)
print("Classes de Teste: ",test_dataset.class_to_idx)





Classes de Treino:  {'apples': 0, 'tomatoes': 1}
Classes de Teste:  {'apples': 0, 'tomatoes': 1}


Após isso vamos criar os **DataLoaders** para carregar os dados e separar em lotes.

O DataLoader é uma classe em PyTorch que cuida de carregar os dados e fornecer lotes de amostras durante o treinamento de um modelo. Ele divide os dados em batches (lotes) e permite que o treinamento seja feito em pedaços menores, em vez de carregar toda a base de dados de uma vez.

- *batch_size*: Define o número de amostras processadas em cada passo de treinamento. Exemplo: 32 imagens por vez.

- *shuffle*: Se for True, embaralha os dados a cada época de treinamento para garantir que o modelo não aprenda padrões artificiais pela ordem dos dados.


In [6]:
# Criar DataLoaders para treinar em batches
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Agora vamos definir uma **rede neural convolucional** usando **torch.nn**. 

As **camadas convolucionais** são um dos componentes fundamentais das redes neurais convolucionais (CNNs), que são amplamente usadas em tarefas de visão computacional, como reconhecimento de imagens, detecção de objetos, e segmentação de imagens. Elas têm a função principal de extrair características ou padrões das imagens, como bordas, texturas e formas, para ajudar o modelo a entender os objetos ou os contextos dentro de uma imagem.

In [7]:
class TomatoVsAppleCNN(nn.Module):
    def __init__(self):
        super(TomatoVsAppleCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(32, 64, 3, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 3, 1, 1)
        
        self.fc1 = nn.Linear(128 * 18 * 18, 128)  # Ajustar o tamanho dependendo do tamanho final
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)  # Saída binária
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = torch.flatten(x, start_dim=1)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))  # Saída entre 0 e 1
        return x

Agora, vamos configurar o treinamento. 

Caso a máquina que esteja rodando tenha uma placa de vídeo Nvidia e ela tem suporte a processamento CUDA, então vamos usar o **torch.cuda.is_available()** caso contrario vamos usar a CPU.

Para fazer o modelo, aplicamos a função de convolução ao dispositivo (no meu caso CUDA).

Definimos a função de perda e o otimizador.
Como temos uma saída binária, usamos de critério o Binary Cross Entropy.

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TomatoVsAppleCNN().to(device)

# Função de perda e otimizador
criterion = nn.BCELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001)

Agora vem a parte divertida, treinar o modelo!

Vamos definir a quantidade de épocas dele, ou seja a quantidade de vezes que o modelo vai passar pelos dados e ajustar o peso de cada um.

In [9]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)

        optimizer.zero_grad()
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        predictions = (outputs > 0.5).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

    accuracy = 100 * correct / total
    print(f"Época [{epoch+1}/{num_epochs}], Perda: {running_loss/len(train_loader):.4f}, Acurácia: {accuracy:.2f}%")


Época [1/10], Perda: 0.7533, Acurácia: 53.40%
Época [2/10], Perda: 0.6547, Acurácia: 61.22%
Época [3/10], Perda: 0.6276, Acurácia: 67.35%
Época [4/10], Perda: 0.5789, Acurácia: 70.07%
Época [5/10], Perda: 0.5725, Acurácia: 69.39%
Época [6/10], Perda: 0.5905, Acurácia: 71.43%
Época [7/10], Perda: 0.5696, Acurácia: 68.37%
Época [8/10], Perda: 0.6190, Acurácia: 71.09%
Época [9/10], Perda: 0.6094, Acurácia: 68.37%
Época [10/10], Perda: 0.5707, Acurácia: 72.11%


Veja que em cada Época, o treinamento foi se aprimorando e algumas vezes foi retrocedendo, isso ocorre porque os pesos foram sendo estabelecidos automaticamente pelo algoritmo.

Após o treinamento, precisamos medir o desempenho do modelo para garantir que ele generaliza bem para novas imagens. Para isso, utilizamos o conjunto de teste e calculamos métricas como acurácia, precisão, recall e F1-score.

A avaliação nos permite verificar quão bem o modelo diferencia maçãs de tomates e identificar possíveis problemas, como overfitting (quando o modelo memoriza os dados de treino e não generaliza bem para novos dados).

In [15]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)
        outputs = model(images).squeeze()
        predictions = (outputs > 0.5).float()
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

accuracy = 100 * correct / total
print(f"Acurácia no conjunto de teste: {accuracy:.2f}%")

Acurácia no conjunto de teste: 70.10%


Caso queira o relatório completo, faça:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

model.eval()
all_predictions = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device, dtype=torch.float32)
        outputs = model(images).squeeze()  # Obtém as previsões
        predictions = (outputs > 0.5).float()  # Converte probabilidades para 0 ou 1
        
        all_predictions.extend(predictions.cpu().numpy())  # Salva previsões
        all_labels.extend(labels.cpu().numpy())  # Salva rótulos reais

accuracy = (sum(1 for p, l in zip(all_predictions, all_labels) if p == l) / len(all_labels)) * 100
precision = precision_score(all_labels, all_predictions)
recall = recall_score(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions)

print(f"Acurácia: {accuracy:.2f}%")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Agora, vamos testar com uma imagem aleatória na internet e ver se nosso modelo está bem treinado para essa função. 

In [31]:
from PIL import Image

def predict_image(img_path, model):
    model.eval()
    transform = transforms.Compose([
        transforms.Resize((150, 150)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    img = Image.open(img_path)
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img).squeeze().item()
        class_label = "Tomate" if output > 0.5 else "Maçã"
        print(f"Predição: {class_label} (Confiança: {output:.2f})")

# Exemplo de uso:
predict_image("C:\\Users\\Matheus\\Documents\\Github Projects\\research_MLs\\projects\\LMs\\dataset\\tomatoes_vs_apples\\test\\apples\\img_p1_84.jpeg", model)
predict_image("C:\\Users\\Matheus\\Documents\\Github Projects\\research_MLs\\projects\\LMs\\dataset\\tomatoes_vs_apples\\test\\apples\\img_p3_86.jpeg", model)

Predição: Maçã (Confiança: 0.36)
Predição: Tomate (Confiança: 0.62)


Aparentemente, nosso modelo não está bem treinado... talvez um FineTunning resolva esse problema.